# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the entire DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the entire DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
loans_df.columns

Index(['payment_history', 'location_parameter', 'stem_degree_score',
       'gpa_ranking', 'alumni_success', 'study_major_code',
       'time_to_completion', 'finance_workshop_score', 'cohort_ranking',
       'total_loan_score', 'financial_aid_score', 'credit_ranking'],
      dtype='object')

In [4]:
loans2_df = loans_df[['payment_history', 'location_parameter','gpa_ranking', 'time_to_completion', 'finance_workshop_score', 'cohort_ranking',
       'total_loan_score', 'financial_aid_score', 'credit_ranking']]
loans2_df.head()

,payment_history,location_parameter,gpa_ranking,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,1.9,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,2.6,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,2.3,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,1.9,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,1.9,34.0,0.9978,3.51,0.56,9.4,0


In [5]:
# Review the data types associated with the columns
loans2_df.dtypes

,0
payment_history,float64
location_parameter,float64
gpa_ranking,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64
financial_aid_score,float64
credit_ranking,int64


In [6]:
# Check the credit_ranking value counts
loans2_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column
y = loans2_df["credit_ranking"].values

# Display a sample of y
y

array([0, 0, 0, ..., 1, 0, 1])

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans2_df.drop(columns=["credit_ranking"]).copy()

# Review the features DataFrame
X.head()

,payment_history,location_parameter,gpa_ranking,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,1.9,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,2.6,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,2.3,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,1.9,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,1.9,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
print(input_nodes)

8


In [12]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_nodes = 1


In [14]:
# Create the Sequential model instance
# adjusted layer code details on node descriptions
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation="tanh"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6)                   │              54 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              21 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79 (316.00 B)

 Trainable params: 79 (316.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5097 - loss: 0.8668
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5218 - loss: 0.8185
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5258 - loss: 0.7856
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5571 - loss: 0.7347
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5493 - loss: 0.7257
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5487 - loss: 0.7133
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5357 - loss: 0.7032
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5598 - loss: 0.6824
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5950 - loss: 0.6692
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6299 - loss: 0.6576
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6451 - loss: 0.6450
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6214 - lo

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# self note ~ why does 'ms/step' display here but not on starter file?

13/13 - 0s - 13ms/step - accuracy: 0.6350 - loss: 0.6230
Loss: 0.6230127215385437, Accuracy: 0.6349999904632568


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [19]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

#nn_imported = tf.keras.models.load_model(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [20]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [21]:
# Make predictions with the test data
# Display a sample of the predictions

predictions = nn_model.predict(X_test,verbose=2)
predictions

13/13 - 0s - 11ms/step


array([[0.6441107 ],
       [0.4515754 ],
       [0.6539334 ],
       [0.575491  ],
       [0.68081367],
       [0.6499798 ],
       [0.6662106 ],
       [0.26274067],
       [0.6530292 ],
       [0.5756469 ],
       [0.61616534],
       [0.61531377],
       [0.47719854],
       [0.66847044],
       [0.60547143],
       [0.5270112 ],
       [0.65447086],
       [0.61054987],
       [0.63406706],
       [0.60157436],
       [0.593891  ],
       [0.6367163 ],
       [0.614442  ],
       [0.653395  ],
       [0.3667614 ],
       [0.6743311 ],
       [0.63576305],
       [0.5766748 ],
       [0.48364764],
       [0.24953076],
       [0.5651369 ],
       [0.6437659 ],
       [0.33224204],
       [0.6473615 ],
       [0.37231618],
       [0.48994684],
       [0.5882801 ],
       [0.5908917 ],
       [0.65428174],
       [0.5810418 ],
       [0.6638647 ],
       [0.2513899 ],
       [0.43483138],
       [0.66408736],
       [0.2547947 ],
       [0.6336937 ],
       [0.5611083 ],
       [0.567

In [22]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,1.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [23]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.69      0.40      0.51       188
           1       0.61      0.84      0.71       212

    accuracy                           0.64       400
   macro avg       0.65      0.62      0.61       400
weighted avg       0.65      0.64      0.61       400



---
## Discuss creating a recommendation system for student loans

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

A system to meaningful results as a recommendation system will require features that provide a snapshot of the candidate’s intangibles which indicates how obligations are treated; e.g. payment history, need base as assets in general are not expected to be sizeable at this stage. More so a person's character to obligations is a higher value indicator of loan satisfaction potential than the popularity and compensation band of a chosen 'industry / major'; the latter may be popular at present but no guarantee to be sustainable in the long term.  

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

The model will be content-based filtering based on feature attributes of the user interactions (loan applicant) such as the 'loan score' and 'payment history'.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

A. Building off prior answer where the current 'chosen' major is attracting high industry demand and the loan candidate graduation / certification completion time to complete has to be weighted.  Should a bias be applied to the completion duration to factor in the approval decision has to be considered.

B. Another concern is a stolen identity validation across a candidate pool when all applicants cannot be screened in-person. As bad actors 'also' leverage available AI tools to counter current screening measures make dataset validation with source traceback of equal importance.
